# Uncertainty KDE support

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
from scipy.spatial import distance
from stats import KDE, GaussianMixture

In [ ]:
gm = GaussianMixture([-1, 1], [.5, .3])
(xpdf,), ypdf = gm.pdf(500, [-5], [5])
dx = np.mean(np.diff(xpdf))

In [ ]:
plt.plot(xpdf, ypdf)

In [ ]:
def total_probability(alpha, xpdf, ypdf):
    total = 0
    for i in range(len(xpdf)-1):
        if ypdf[i] > alpha and ypdf[i+1] > alpha:
            total += (xpdf[i+1] - xpdf[i]) * (ypdf[i] + ypdf[i+1]) / 2
        elif ypdf[i] > alpha:
            total += (ypdf[i]+alpha)*(ypdf[i]-alpha)*(xpdf[i+1]-xpdf[i])/(2*(ypdf[i]-ypdf[i+1]))
        elif ypdf[i+1] > alpha:
            total += (ypdf[i+1]+alpha)*(ypdf[i+1]-alpha)*(xpdf[i+1]-xpdf[i])/(2*(ypdf[i+1]-ypdf[i]))
    return total

In [ ]:
def f(alpha, level, xpdf, ypdf):
    return (total_probability(alpha, xpdf, ypdf) - level)**2

In [ ]:
result = minimize(f, 0.1, args=(.9, xpdf, ypdf))
alpha = result.x

In [ ]:
set1 = np.atleast_2d(xpdf[ypdf > alpha]).T

In [ ]:
np.random.seed(0)
n = 40
nrepeat = 100

distances = np.zeros(nrepeat)
for i in range(nrepeat):
    x = gm.generate_samples(n)
    k = KDE(x)
    k.set_bandwidth(.3)
    ykde = k.score_samples(xpdf)
    r = minimize(f, 0.1, args=(.9, xpdf, ykde))
    set2 = np.atleast_2d(xpdf[ykde > r.x]).T
    distances[i] = distance.directed_hausdorff(set1, set2)[0]

In [ ]:
np.mean(distances)

In [ ]:
np.std(distances)

In [ ]:
np.random.seed(0)
x = gm.generate_samples(n)
k = KDE(x)
k.set_bandwidth(.3)
ykde = k.score_samples(xpdf)
r = minimize(f, 0.1, args=(.9, xpdf, ykde))
set2 = np.atleast_2d(xpdf[ykde > r.x]).T

distances2 = np.zeros(nrepeat)
for i in range(nrepeat):
    k = KDE(np.random.choice(x[:, 0], len(x)))
    k.set_bandwidth(.3)
    ykde = k.score_samples(xpdf)
    r = minimize(f, 0.1, args=(.9, xpdf, ykde))
    set3 = np.atleast_2d(xpdf[ykde > r.x]).T
    distances2[i] = distance.directed_hausdorff(set2, set3)[0]

In [ ]:
np.mean(distances2)

In [ ]:
np.std(distances2)